In [ ]:
#%%jupyter_template init
import os, sys; sys.path.insert(0, os.path.realpath('..'))
from jupyter_template import magic
magic.init(lambda _=globals: _())

# Single Cell Enrichment

We prepare single cell data, computing clusters, differential expression, and enrichment analysis.

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import plotly.express as px
import scipy.sparse as sp_sparse
import seaborn as sns
from geode import chdir
from sklearn.preprocessing import StandardScaler
from IPython.display import display
from umap import UMAP
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from collections import OrderedDict
from sklearn.metrics import silhouette_score

In [ ]:
# TODO: move some of these functions to maayanlab_bioinformatics package?

def merge(left, *rights, **kwargs):
  ''' Helper function for many trivial (index based) joins
  '''
  merged = left
  for right in rights:
    merged = pd.merge(left=merged, left_index=True, right=right, right_index=True, **kwargs)
  return merged


def log2Normalize(mat):
  import pandas as pd
  import numpy as np
  if isinstance(mat, pd.DataFrame):
    return pd.DataFrame(np.log2(mat+1), index=mat.index, columns=mat.columns)
  else:
    return np.log2(mat+1)
  
def zscoreNormalize(mat):
  import pandas as pd
  from scipy.stats import zscore
  if isinstance(mat, pd.DataFrame):
    return pd.DataFrame(zscore(mat, axis=0), index=mat.index, columns=mat.columns)
  else:
    return zscore(mat, axis=0)

def quantileNormalize_np(mat):
  import numpy as np
  # sort vector in np (reuse in np)
  sorted_vec = np.sort(mat, axis=0)
  # rank vector in np (no dict necessary)
  rank = sorted_vec.mean(axis=1)
  # construct quantile normalized matrix
  return np.array([
    [
      rank[i]
      for i in np.searchsorted(sorted_vec[:, c], mat[:, c])
    ] for c in range(mat.shape[1])
  ]).T

def quantileNormalize_pd(df):
  import pandas as pd
  return pd.DataFrame(
    quantileNormalize_np(df.values),
    index=df.index,
    columns=df.columns,
  )

def quantileNormalize(arg):
  ''' Perform quantile normalization on the values of a matrix of dataframe
  '''
  import pandas as pd
  import numpy as np
  if isinstance(arg, pd.DataFrame):
    return quantileNormalize_pd(arg)
  elif isinstance(arg, np.ndarray):
    return quantileNormalize_np(arg)
  else:
    raise Exception('quantileNormalize: Unrecognized argument type (`{}`)'.format(type(arg)))

def enrichr_link_from_genes(genes, description='', enrichr_link='https://amp.pharm.mssm.edu/Enrichr'):
  ''' Functional access to Enrichr API
  '''
  import time, requests
  time.sleep(1)
  resp = requests.post(enrichr_link + '/addList', files={
    'list': (None, '\n'.join(genes)),
    'description': (None, description),
  })
  if resp.status_code != 200:
    raise Exception('Enrichr failed with status {}: {}'.format(
      resp.status_code,
      resp.text,
    ))
  # wait a tinybit before returning link (backoff)
  time.sleep(1)
  result = resp.json()
  return dict(result, link=enrichr_link + '/enrich?dataset=' + resp.json()['shortId'])

def enrichr_get_top_results(userListId, bg, enrichr_link='https://amp.pharm.mssm.edu/Enrichr'):
  import time, requests
  time.sleep(1)
  resp = requests.get(enrichr_link + '/enrich?userListId={}&backgroundType={}'.format(userListId, bg))
  if resp.status_code != 200:
    raise Exception('Enrichr failed with status {}: {}'.format(
      resp.status_code,
      resp.text,
    ))
  time.sleep(1)
  return pd.DataFrame(resp.json()[bg], columns=['rank', 'term', 'pvalue', 'zscore', 'combinedscore', 'overlapping_genes', 'adjusted_pvalue', '', ''])

def diffExpression(controls, cases):
  assert controls.shape[0] == cases.shape[0], "Must have the same number of genes"
  genes = np.array(controls.index)
  n_genes = genes.shape[0]
  # Compute characteristic direction
  results = pd.DataFrame(
    data=chdir(
      pd.concat([controls.loc[genes, :], cases.loc[genes, :]], axis=1).values,
      np.array(
        [1]*controls.shape[1] + [2]*cases.shape[1]
      ),
      # genes
      genes,
      # gamma
      0.5,
      sort=False,
      calculate_sig=False,
    ),
    columns=['CD-coefficient', 'index'],
  )
  results.index = results['index']
  return results.drop('index', axis=1)

def upDownFromDiffExpression(expr, top_genes):
  filtered_expr = expr.loc[expr.abs().sort_values('CD-coefficient', ascending=False)[:top_genes].index]
  return {
    'up': list(filtered_expr[filtered_expr > 0].dropna().index),
    'down': list(filtered_expr[filtered_expr < 0].dropna().index),
  }


## Step 1. Configure analysis

In [ ]:
%%jupyter_template code_exec

# the archive containing the single cell data
#  note that the expectation is a directory of the form
#  /{archive}.zip/xxx_xxxx_20190101_xxx_xx_ReplicateX/filtered_feature_bc_matrix
#  /{archive}.zip/...

# TODO: can we use the format that comes out of the actual machine?
# TODO: are there other formats?
{% do SectionField(
    name='INPUT',
    label='Upload your single-cell data',
    description='As a .zip archive containing your single-cell data in a format ready for Seurat',
) %}

archive = {{ FileField(
    name='zip_file',
    label='zip file containing CellRanger output',
    default='scEnrichr.py',
    section='INPUT',
) }}

{% do SectionField(
    name='CONFIG',
    label='Configuration',
    description='Configure various parameters for the analysis',
) %}

# The number of 'top' genes to use for differential expression
top_n_genes = {{ IntField(
    name='top_n_genes',
    label='Number of Genes',
    description='The number of \'top\' genes to use for differential expression',
    default=250,
    min=100,
    max=1000,
    section='CONFIG',
) }}

# The number of 'top' results to keep from enrichment analysis
top_n_results = {{ IntField(
    name='top_n_results',
    label='Number of Top Enrichment Results',
    description='The number of \'top\' results to keep from enrichment analysis',
    default=5,
    min=1,
    max=100,
    section='CONFIG',
) }}

# TODO: add enrichr libraries as categories as fields
useful_libs = OrderedDict([
  ('cell_type', ['Human_Gene_Atlas', 'Mouse_Gene_Atlas', 'ARCHS4_Tissues']),
  ('pathways', ['WikiPathways_2019_Mouse', 'WikiPathways_2019_Human']),
  ('transcription', ['ARCHS4_TFs_Coexp', 'ENCODE_and_ChEA_Consensus_TFs_from_ChIP-X']),
])

## Step 2. Fetch and extract data

In [ ]:
%%jupyter_template code_exec

directory = os.path.basename(archive)

if not os.path.exists(directory):
    if not os.path.exists(archive):
        import urllib.request
        urllib.request.urlretrieve('https://jupy-template.cloud/{{ _session }}/' + archive, archive)

    from zipfile import ZipFile
    with ZipFile(file, 'r') as zipObj:
        zipObj.extractall(directory)

## Step 3. Load in data

In [ ]:
def load_suerat_files(base_dir):
  ''' Helper function for loading in files made for seurat
  '''
  import pandas as pd
  import scipy.sparse as sp_sparse
  df_barcodes = pd.read_csv(
    os.path.join(base_dir, 'barcodes.tsv.gz'),
    index_col=0,
    header=None,
    sep='\t',
  )
  df_features = pd.read_csv(
    os.path.join(base_dir, 'features.tsv.gz'),
    header=None,
    names=['symbol', 'type'],
    index_col=0,
    sep='\t',
  )
  matrix = pd.read_csv(
    os.path.join(base_dir, 'matrix.mtx.gz'),
    header=None,
    names=['indices', 'indptr', 'data'],
    skiprows=2,
    sep=' ',
  )
  csc_matrix = sp_sparse.csc_matrix(
    (
      matrix['data'].values,
      (
        matrix['indices'].values - 1, # 0 based indexing
        matrix['indptr'].values - 1,  # 0 based indexing
      )
    ),
  )
  df_expression = pd.DataFrame(csc_matrix.todense())
  df_expression.index = df_features.index
  df_expression.columns = df_barcodes.index
  return df_features, df_barcodes, df_expression


In [ ]:
all_df_features = []
all_df_barcodes = []
all_df_expression = []

for ind, file in enumerate(files):
  df_features, df_barcodes, df_expression = load_suerat_files(os.path.join(base_path, file))
  df_barcodes['barcode'] = df_barcodes.index
  df_barcodes['file'] = f"File {ind}"
  df_barcodes.index = df_barcodes.index.map(lambda s, ind=ind: f"{ind}:{s}")
  df_expression.columns = df_barcodes.index
  all_df_features.append(df_features)
  all_df_barcodes.append(df_barcodes)
  all_df_expression.append(df_expression)

df_features = merge(*all_df_features, how='left', suffixes=('', '_')).drop(['symbol_', 'type_'], axis=1)
df_barcodes = pd.concat(all_df_barcodes)
df_expression = merge(*all_df_expression)

## Step 4. Map transcripts to Genes

### Get NCBI Gene information

In [ ]:
# TODO: allow organism to be configured
ncbi = pd.read_csv('ftp://ftp.ncbi.nih.gov/gene/DATA/GENE_INFO/Mammalia/Homo_sapiens.gene_info.gz', sep='\t')
# Ensure nulls are treated as such
ncbi = ncbi.applymap(lambda v: float('nan') if type(v) == str and v == '-' else v)
# Break up lists
split_list = lambda v: v.split('|') if type(v) == str else []
ncbi['dbXrefs'] = ncbi['dbXrefs'].apply(split_list)
ncbi['Synonyms'] = ncbi['Synonyms'].apply(split_list)
ncbi['LocusTag'] = ncbi['LocusTag'].apply(split_list)
ncbi['Other_designations'] = ncbi['Other_designations'].apply(split_list)

# Map existing entities to NCBI Genes
ncbi_lookup = {
  sym.upper(): row['Symbol'].upper()
  for _, row in ncbi.iterrows()
  for sym in [row['Symbol']] + row['Synonyms']
}

### Select transcripts with highest variance corresponding to genes

In [ ]:
df_transcript_genes = merge(
  df_expression.var(axis=1).to_frame('var'),
  df_features[['symbol']].applymap(lambda s: str(ncbi_lookup.get(s.upper())))
).groupby('symbol')['var'].idxmax().reset_index()
df_transcript_genes.index = df_transcript_genes['var']
df_transcript_genes = df_transcript_genes.drop('var', axis=1)
df_transcript_genes

### Obtain a gene expression matrix

In [ ]:
df_gene_expression = df_expression.loc[df_transcript_genes.index]
df_gene_expression.index = df_transcript_genes['symbol']

## Step 5. Normalize Gene Expression Matrix

### Review existing library size and distribution

In [ ]:
df_library_size = pd.DataFrame(
    {
        'n_reads': df_gene_expression[df_gene_expression > 0].count(),
        'log_n_reads': np.log2(df_gene_expression[df_gene_expression > 0].count() + 1),
        'n_expressed_genes': df_gene_expression.sum(),
    }
).sort_values('n_reads', ascending=False)

display(df_library_size.head())
sns.distplot(df_gene_expression.iloc[0, :]); plt.show()
sns.distplot(df_gene_expression.iloc[:, 0]); plt.show()

### Perform normalization

In [ ]:
# TODO: make configurable

# Seurat.NormalizeData: log normalize
df_gene_expression_norm = np.log(df_gene_expression + 1)
# Seurat.FindVariableFeatures: select top 2000 most variable features
df_gene_expression_norm = df_gene_expression_norm.loc[df_gene_expression.var(axis=1).sort_values()[-2000:].index]
# Seurat.ScaleData: zero mean & unit variance
df_gene_expression_norm = pd.DataFrame(StandardScaler().fit_transform(df_gene_expression_norm), index=df_gene_expression_norm.index, columns=df_gene_expression_norm.columns)
# df_gene_expression_norm = zscoreNormalize(df_gene_expression_norm)
# df_gene_expression_norm = quantileNormalize(df_gene_expression_norm)

display(df_gene_expression_norm.head())

### Review normalized count distributions

In [ ]:
# TODO: potentially evaluate kurtosis and warn about problems with normalization
sns.distplot(df_gene_expression_norm.iloc[0, :]); plt.show()
sns.distplot(df_gene_expression_norm.iloc[:, 0]); plt.show()

## Step 6. Dimensionality Reduction & Visualization

### PCA

In [ ]:
gene_expression_norm_pca = PCA(random_state=42)
gene_expression_norm_pca.fit(df_gene_expression_norm.values.T)
df_gene_expression_norm_pca = pd.DataFrame(
    gene_expression_norm_pca.transform(df_gene_expression_norm.values.T),
    index=df_gene_expression_norm.T.index
)
df_gene_expression_norm_pca.columns = [
    f'PCA-{c} ({r:.3f})'
    for c, r in zip(df_gene_expression_norm_pca.columns, gene_expression_norm_pca.explained_variance_ratio_)
]

In [ ]:
px.scatter(
  merge(
    df_gene_expression_norm_pca,
    df_barcodes,
    df_library_size,
  ),
  x=df_gene_expression_norm_pca.columns[0],
  y=df_gene_expression_norm_pca.columns[1],
  size='n_reads',
  size_max=8,
  symbol='file',
  hover_data=[df_gene_expression_norm.columns],
)

### UMAP

In [ ]:
# TODO: make configurable
gene_expression_norm_umap = UMAP(
  random_state=42,
  n_components=2,
  n_neighbors=30,
  metric='cosine',
  min_dist=0.3,
)
gene_expression_norm_umap.fit(df_gene_expression_norm_pca.iloc[:, :10].values)

df_gene_expression_norm_umap = pd.DataFrame(
  gene_expression_norm_umap.transform(df_gene_expression_norm_pca.iloc[:, :10].values),
  columns=['UMAP-1', 'UMAP-2'],
  index=df_gene_expression_norm_pca.index,
)

In [ ]:
px.scatter(
  merge(
    df_gene_expression_norm_umap,
    df_barcodes,
    df_library_size,
  ),
  x=df_gene_expression_norm_umap.columns[0],
  y=df_gene_expression_norm_umap.columns[1],
  size='n_reads',
  size_max=8,
  symbol='file',
  hover_data=[df_gene_expression_norm.columns],
)

## Step 7. Silhouette Cluster Analysis

In [ ]:
silhouette_scores = {}
for n in range(2, 25):
    np.random.seed(0)
    y_pred = KMeans(n_clusters=n, random_state=42).fit_predict(df_gene_expression_norm_umap.values)
    silhouette_scores[n] = silhouette_score(df_gene_expression_norm_umap.values, y_pred, metric='cosine')

silhouette_scores = pd.DataFrame([
    {'N Clusters': k, 'Silhouette Score': v}
    for k, v in silhouette_scores.items()
])
best = silhouette_scores.sort_values('Silhouette Score').iloc[-1]
silhouette_scores

In [ ]:
plt.plot(silhouette_scores['N Clusters'], silhouette_scores['Silhouette Score'])
plt.scatter([best['N Clusters']], [best['Silhouette Score']], label='Best')
plt.legend()
plt.title('Cluster size selection')
plt.ylabel('Silhouette Score')
plt.xlabel('Number of Clusters')
plt.show()

In [ ]:
km = KMeans(n_clusters=int(best['N Clusters']), random_state=42)
df_gene_expression_norm_km = pd.DataFrame({
    'Cluster': [
        f'Cluster {c}'
        for c in km.fit_predict(df_gene_expression_norm_umap.values)
    ]
}, index=df_gene_expression_norm_umap.index)

### PCA with Clusters

In [ ]:
px.scatter(
  merge(
    df_gene_expression_norm_pca,
    df_gene_expression_norm_km,
    df_barcodes,
    df_library_size,
  ),
  x=df_gene_expression_norm_pca.columns[0],
  y=df_gene_expression_norm_pca.columns[1],
  size='n_reads',
  size_max=8,
  symbol='file',
  color='Cluster',
  hover_data=[df_gene_expression_norm.columns],
)

### UMAP with Clusters

In [ ]:
px.scatter(
  merge(
    df_gene_expression_norm_umap,
    df_gene_expression_norm_km,
    df_barcodes,
    df_library_size,
  ),
  x=df_gene_expression_norm_umap.columns[0],
  y=df_gene_expression_norm_umap.columns[1],
  size='n_reads',
  size_max=8,
  symbol='file',
  color='Cluster',
  hover_data=[df_gene_expression_norm.columns],
)

## Step 8. Differential Expression

We perform differential expression for each cluster in a one vs rest fashion.

In [ ]:
# Perform differential expression for each cluter
top_genes = {}
for cluster, samples in df_gene_expression_norm_km.groupby('Cluster'):
  top_genes[cluster] = upDownFromDiffExpression(
    diffExpression(
      # expression outside of this cluster
      df_gene_expression_norm.loc[:, df_gene_expression_norm.columns.difference(samples.index)],
      # expression in this cluster
      df_gene_expression_norm.loc[:, samples.index],
    ),
    top_n_genes,
  )

pd.DataFrame(top_genes)

## Step 9. Enrichment Analysis

### We submit differentially expressed genes to Enrichr.

In [ ]:
# Get Enrichr links for each cluster
enrichr_links = {}

for cluster, genes in top_genes.items():
  up_link, dn_link = None, None
  if len(genes['up']):
    up_link = enrichr_link_from_genes(sorted(genes['up']), 'cluster %s up' % (cluster))
    # display_link_inline(up_link['link'])
  else:
    print('cluster %s up: empty' % (cluster))
  if len(genes['down']):
    dn_link = enrichr_link_from_genes(sorted(genes['down']), 'cluster %s down' % (cluster))
    # display_link_inline(dn_link['link'])
  else:
    print('cluster %s down: empty' % (cluster))
  enrichr_links[cluster] = {
    'up': up_link,
    'down': dn_link,
  }

pd.DataFrame(enrichr_links)

### Grab top results from Enrichr results

In [ ]:
# Grab top results for each cluster
all_results = []
for cluster, links in enrichr_links.items():
  for link_type, link in links.items():
    if link is None:
      continue
    for category, libraries in useful_libs.items():
      for library in libraries:
        try:
          results = enrichr_get_top_results(link['userListId'], library).sort_values('pvalue').iloc[:top_n_results]
          results['link'] = link['link']
          results['library'] = library
          results['category'] = category
          results['direction'] = link_type
          results['cluster'] = cluster
          all_results.append(results)
        except:
          print('{}: {} {} {} cluster {} failed, continuing'.format(link, library, category, link_type, cluster))

df_all_results = pd.concat(all_results)
df_all_results

## Step 10. Export results for scEnrichr Dashboard

In [ ]:
g = merge(df_gene_expression_norm_km, df_gene_expression_norm_pca)
g.index.rename('Barcode', inplace=True)
g.reset_index().to_csv(
  os.path.join(output, 'df_pca.tsv'),
  sep='\t',
  index=None,
)

g = merge(df_gene_expression_norm_km, df_gene_expression_norm_umap)
g.index.rename('Barcode', inplace=True)
g.reset_index().to_csv(
  os.path.join(output, 'df_umap.tsv'),
  sep='\t',
  index=None,
)

df_all_results.to_csv(
  os.path.join(output, 'df_enrich.tsv'),
  sep='\t',
  index=None,
)

The files are now available for download and for display with the scEnrichr Dashboard:

- [df_pca.tsv](./df_pca.tsv)
- [df_umap.tsv](./df_umap.tsv)
- [df_enrich.tsv](./df_enrich.tsv)


**[View Dashboard](./dashboard/)** (TODO)